# add groups

In [1]:
# mouse thymus
import scanpy as sc
import pandas as pd

ad_protein = sc.read_h5ad('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/adata_ADT.h5ad')

df = pd.read_csv('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/SpaMV_z.csv', index_col=0)
df['protein_group'] = df.idxmax(axis=1)
df.loc[df['protein_group'].str.contains('Transcriptomics', case=False, na=False), 'protein_group'] = 'none'

ad_protein.obs['protein_group'] = df['protein_group'].reindex(ad_protein.obs.index).fillna('none')
# ad_protein.write('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yang/Dimension reduction results/Mouse Embryo results/adata_ADT_addgroups.h5ad')

In [2]:
ad_st = sc.read_h5ad('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/adata_RNA.h5ad')

df = pd.read_csv('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/SpaMV_z.csv', index_col=0)
df['transcriptomics_group'] = df.idxmax(axis=1)
df.loc[df['transcriptomics_group'].str.contains('Proteomics', case=False, na=False), 'transcriptomics_group'] = 'none'

ad_st.obs['transcriptomics_group'] = df['transcriptomics_group'].reindex(ad_st.obs.index).fillna('none')
# ad_st.write('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/adata_RNA_addgroups.h5ad')

In [7]:
ad_st

AnnData object with n_obs × n_vars = 4646 × 23960
    obs: 'orig.ident', 'x', 'y', 'transcriptomics_group'
    obsm: 'spatial'

In [9]:
ad_protein.var_names

Index(['mouse_CD5', 'mouse_CD4', 'mouse_CD68', 'mouse_rat_CD29', 'mouse_CD8a',
       'mouse_CD3', 'mouse_human_CD44', 'mouse_CD90_2', 'mouse_rat_human_CD27',
       'mouse_CD11c', 'mouse_CD31', 'mouse_F4_80', 'Rat_IgG2a',
       'mouse_human_CD45R_B220', 'mouse_CD19', 'mouse_human_CD11b',
       'Mouse_IgG2a', 'mouse_Siglec_H', 'mouse_CD169_Siglec-1'],
      dtype='object')

# select overlapping genes

In [13]:
gene_names = [
    "Cd5",
    "Cd4",
    "Cd68",
    "Itgb1",
    "Cd8a",
    "Cd3",  # CD3 complex
    "Cd44",
    "Thy1",
    "Cd27",
    "Itgax",
    "Pecam1",
    "Adgre1",
    "None",  # Rat_IgG2a – isotype control
    "Ptprc",
    "Cd19",
    "Itgam",
    "None",  # Mouse_IgG2a – isotype control
    "Siglech",
    "Siglec1"
]

ad_protein.var_names = gene_names

In [14]:
# Convert var_names to a regular pandas Index if it's not already
ad_st.var_names = pd.Index(ad_st.var_names)

# Step 1: Create a boolean mask for non-None entries
valid_mask = [v is not None and v != "None" for v in ad_st.var_names]

# Step 2: Subset AnnData to keep only valid var_names
ad_st = ad_st[:, valid_mask].copy()

# Step 1: Ensure var_names are unique in both AnnData objects
ad_st.var_names_make_unique()
ad_protein.var_names_make_unique()

# Step 2: Find common genes
common_genes = ad_protein.var_names.intersection(ad_st.var_names)

# Step 3: Subset
ad_st_subset = ad_st[:, common_genes].copy()
ad_protein_subset = ad_protein[:, common_genes].copy()

In [16]:
ad_st_subset

AnnData object with n_obs × n_vars = 4646 × 16
    obs: 'orig.ident', 'x', 'y', 'transcriptomics_group'
    obsm: 'spatial'

In [17]:
ad_st_subset.write("/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/adata_RNA_addgroups_match.h5ad")
ad_protein_subset.write("/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/adata_ADT_addgroups_match.h5ad")

# to GSVA input format

In [20]:
import numpy as np
import os


# adata_subset = sc.read_h5ad('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yang/Dimension reduction results/9_Mouse_Thymus/adata_ADT_addgroups_match.h5ad')
# out_path = '/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yang/mouse_thymus/mouse_thymus_protein_match'
adata_subset = sc.read_h5ad('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/adata_RNA_addgroups_match.h5ad')
out_path = '/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/mouse_thymus_transcriptomics_match'

os.makedirs(out_path, exist_ok=True)


# Get a boolean mask: True for duplicates (excluding the first one)
duplicates_mask = adata_subset.var.index.duplicated(keep='first')

# Filter out duplicated genes from var and corresponding columns in X
adata_subset._inplace_subset_var(~duplicates_mask)



# Extract the expression matrix, sample data (phenotype), and feature data (genes)
expression_matrix = adata_subset.X  # gene expression matrix (usually a sparse matrix)
feature_data = adata_subset.var.astype(str)  # features (genes)
phenotype_data = adata_subset.obs.astype(str)  # observations (samples)

# Convert to DataFrame for easy handling
expression_matrix_df = pd.DataFrame(expression_matrix.toarray())

# Create a new index for the samples (1 to length of phenotype_data)
new_index = [f"cell{i+1}" for i in range(len(phenotype_data))]

# Create a mapping relation between the original index and the new index
index_mapping = pd.DataFrame({
    'Original Index': phenotype_data.index,
    'New Index': new_index
})

# Update the index of expression_matrix_df
expression_matrix_df.index = new_index  # sample names
phenotype_data.index = new_index

# Also update the columns (gene names)
expression_matrix_df.columns = feature_data.index.astype(str)  # gene names

# Save the updated DataFrame and the mapping relation as CSV
expression_matrix_df.T.to_csv(os.path.join(out_path, 'expression_matrix.csv'))
phenotype_data.to_csv(os.path.join(out_path, 'phenotype_data.csv'))
feature_data.to_csv(os.path.join(out_path, 'feature_data.csv'))
index_mapping.to_csv(os.path.join(out_path, 'index_mapping.csv'), index=False)

In [21]:
import numpy as np
import os


adata_subset = sc.read_h5ad('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/adata_ADT_addgroups_match.h5ad')
out_path = '/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/mouse_thymus_protein_match'
# adata_subset = sc.read_h5ad('/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/adata_RNA_addgroups_match.h5ad')
# out_path = '/maiziezhou_lab2/yunfei/Projects/Spinal_MERFISH/GSVA/data_in/yangv2/Mouse Thymus results (RNA+Protein)/mouse_thymus_transcriptomics_match'

os.makedirs(out_path, exist_ok=True)


# Get a boolean mask: True for duplicates (excluding the first one)
duplicates_mask = adata_subset.var.index.duplicated(keep='first')

# Filter out duplicated genes from var and corresponding columns in X
adata_subset._inplace_subset_var(~duplicates_mask)



# Extract the expression matrix, sample data (phenotype), and feature data (genes)
expression_matrix = adata_subset.X  # gene expression matrix (usually a sparse matrix)
feature_data = adata_subset.var.astype(str)  # features (genes)
phenotype_data = adata_subset.obs.astype(str)  # observations (samples)

# Convert to DataFrame for easy handling
expression_matrix_df = pd.DataFrame(expression_matrix.toarray())

# Create a new index for the samples (1 to length of phenotype_data)
new_index = [f"cell{i+1}" for i in range(len(phenotype_data))]

# Create a mapping relation between the original index and the new index
index_mapping = pd.DataFrame({
    'Original Index': phenotype_data.index,
    'New Index': new_index
})

# Update the index of expression_matrix_df
expression_matrix_df.index = new_index  # sample names
phenotype_data.index = new_index

# Also update the columns (gene names)
expression_matrix_df.columns = feature_data.index.astype(str)  # gene names

# Save the updated DataFrame and the mapping relation as CSV
expression_matrix_df.T.to_csv(os.path.join(out_path, 'expression_matrix.csv'))
phenotype_data.to_csv(os.path.join(out_path, 'phenotype_data.csv'))
feature_data.to_csv(os.path.join(out_path, 'feature_data.csv'))
index_mapping.to_csv(os.path.join(out_path, 'index_mapping.csv'), index=False)